# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6731821550576677072
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 3709404683481136449
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2024-02-20 17:18:44.012225: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-20 17:18:44.012250: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 데이터 셋 읽어오기

In [13]:
df = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [14]:
df = df.drop(['customer_idx', 'product_subcategory', 'product_modelname', 'lead_owner', 'business_subarea',
                          'customer_country.1'], axis = 1)
df

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,...,product_category,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,End-Customer,Enterprise,NaN,NaN,NaN,NaN,...,multi-split,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
1,1.00,/PH-00/Philippines,AS,0.066667,End-Customer,Enterprise,12.0,NaN,NaN,NaN,...,multi-split,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
2,1.00,/Kolkata /India,AS,0.088889,End-Customer,Enterprise,144.0,NaN,NaN,NaN,...,single-split,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
3,1.00,/Bhubaneswar/India,AS,0.088889,End-Customer,Enterprise,NaN,NaN,NaN,NaN,...,vrf,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
4,1.00,/Hyderabad/India,AS,0.088889,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,multi-split,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,/Sląskie/Poland,AS,NaN,End Customer,SMB,NaN,NaN,NaN,NaN,...,vrf,entry level,LGEPL,3 months ~ 6 months,0,0,0.000026,0.028777,public facility,False
59295,0.75,/Bogotá DC /Colombia,AS,0.040000,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,etc.,trainee,LGECB,9 months ~ 1 year,0,0,0.000026,0.028777,public facility,False
59296,0.75,/Pisco/Peru,AS,0.040000,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,single-split,entry level,LGEPR,less than 3 months,0,0,0.000026,0.028777,public facility,False
59297,1.00,/santa cruz bolivia/Peru,AS,0.040000,NaN,Enterprise,NaN,NaN,NaN,NaN,...,etc.,others,LGEPR,more than a year,0,0,0.000026,0.028777,public facility,False


In [61]:
df_test = df_test.drop(['id'], axis = 1)
df_test

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,...,product_category,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,is_converted
0,0.00,/ / Brazil,ID,0.073248,End Customer,Enterprise,53.0,NaN,NaN,NaN,...,NaN,none,LGESP,NaN,1,0,0.001183,0.049840,retail,True
1,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,End Customer,SMB,NaN,NaN,NaN,NaN,...,NaN,none,LGEUS,NaN,0,0,0.000013,NaN,transportation,True
2,1.00,/ / U.A.E,ID,NaN,Specifier/ Influencer,SMB,NaN,NaN,NaN,NaN,...,hospital tv,manager,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,False
3,0.50,/ Madison / United States,ID,0.118644,NaN,Enterprise,NaN,NaN,NaN,NaN,...,NaN,none,LGEUS,more than a year,0,0,0.001183,0.049840,retail,False
4,1.00,/ Sao Paulo / Brazil,ID,0.074949,End Customer,Enterprise,2.0,1.0,NaN,1.0,...,led signage,others,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,0.50,/São Paulo/Brazil,AS,NaN,NaN,Enterprise,10.0,NaN,NaN,NaN,...,NaN,associate/analyst,LGESP,NaN,0,0,NaN,NaN,NaN,False
5267,0.25,General / / United States,IT,NaN,NaN,Enterprise,0.0,NaN,NaN,NaN,...,NaN,none,LGEUS,NaN,0,0,NaN,NaN,NaN,False
5268,0.75,/ OURO BRANCO / Brazil,AS,NaN,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,multi-split,manager,LGESP,less than 3 months,0,0,NaN,NaN,NaN,True
5269,0.00,/ / Germany,IT,NaN,End Customer,SMB,NaN,NaN,NaN,NaN,...,cloud device,none,LGEDG,NaN,0,0,NaN,NaN,NaN,False


In [15]:
df = df.drop(['bant_submit', 'com_reg_ver_win_rate', 'historical_existing_cnt', 'id_strategic_ver',
                          'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu'], axis = 1)
df.head()

,customer_country,business_unit,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,customer_position,response_corporate,expected_timeline,business_area,is_converted
0,/Quezon City/Philippines,AS,End-Customer,Enterprise,NaN,purchasing,62,Quotation or purchase consultation,multi-split,entry level,LGEPH,less than 3 months,corporate / office,True
1,/PH-00/Philippines,AS,End-Customer,Enterprise,NaN,media and communication,96,Quotation or purchase consultation,multi-split,ceo/founder,LGEPH,less than 3 months,corporate / office,True
2,/Kolkata /India,AS,End-Customer,Enterprise,NaN,engineering,56,Product Information,single-split,partner,LGEIL,less than 3 months,corporate / office,True
3,/Bhubaneswar/India,AS,End-Customer,Enterprise,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,ceo/founder,LGEIL,less than 3 months,corporate / office,True
4,/Hyderabad/India,AS,Specifier/ Influencer,Enterprise,NaN,consulting,97,Quotation or purchase consultation,multi-split,partner,LGEIL,less than 3 months,corporate / office,True


In [63]:
df_test = df_test.drop(['bant_submit', 'com_reg_ver_win_rate', 'historical_existing_cnt', 'id_strategic_ver',
                          'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu'], axis = 1)
df_test.head()

,customer_country,business_unit,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,customer_position,response_corporate,expected_timeline,business_area,is_converted
0,/ / Brazil,ID,End Customer,Enterprise,NaN,consulting,3,NaN,NaN,none,LGESP,NaN,retail,True
1,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,End Customer,SMB,NaN,NaN,23,Quotation or Purchase Consultation,NaN,none,LGEUS,NaN,transportation,True
2,/ / U.A.E,ID,Specifier/ Influencer,SMB,NaN,information technology,138,Quotation or Purchase Consultation,hospital tv,manager,LGEGF,less than 3 months,hospital & health care,False
3,/ Madison / United States,ID,NaN,Enterprise,NaN,sales,3,Quotation or Purchase Consultation,NaN,none,LGEUS,more than a year,retail,False
4,/ Sao Paulo / Brazil,ID,End Customer,Enterprise,1.0,engineering,107,Quotation or Purchase Consultation,led signage,others,LGESP,less than 3 months,corporate / office,False


In [16]:
print(df['customer_country'].unique()[:10])
print(df['customer_country'].unique()[-10:])

['/Quezon City/Philippines' '/PH-00/Philippines' '/Kolkata /India'
 '/Bhubaneswar/India' '/Hyderabad/India' '/Abuja/Nigeria'
 '/Jeddah, KSA/Saudi Arabia' '/Guwahati/India' '/Cebu city/Philippines'
 '/hauz khas,delhi/India']
['/Itaú de Minas - Minas Gerais/Brazil' '/SP/ Jundiai/Brazil'
 '/Sergipe/Aracaju/Brazil' '/Estado de Mexico /Mexico'
 '/Baturaja/Indonesia' '/Sląskie/Poland' '/Bogotá DC /Colombia'
 '/Pisco/Peru' '/santa cruz bolivia/Peru' '/paris/France']


# customer_country

In [17]:
df['customer_nation'] = df['customer_country'].str.split('/').str[-1]

import pycountry

def validate_country(data):
    try:
        return pycountry.countries.lookup(data).name
    except LookupError:
        return pd.NA   # 국가이름 아닐시 nan으로 반환
df['customer_nation'] = df['customer_nation'].apply(validate_country)
df_country = df['customer_nation']
df['customer_country'] = df_country
df.head()

,customer_country,business_unit,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,customer_position,response_corporate,expected_timeline,business_area,is_converted,customer_nation
0,Philippines,AS,End-Customer,Enterprise,NaN,purchasing,62,Quotation or purchase consultation,multi-split,entry level,LGEPH,less than 3 months,corporate / office,True,Philippines
1,Philippines,AS,End-Customer,Enterprise,NaN,media and communication,96,Quotation or purchase consultation,multi-split,ceo/founder,LGEPH,less than 3 months,corporate / office,True,Philippines
2,India,AS,End-Customer,Enterprise,NaN,engineering,56,Product Information,single-split,partner,LGEIL,less than 3 months,corporate / office,True,India
3,India,AS,End-Customer,Enterprise,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,ceo/founder,LGEIL,less than 3 months,corporate / office,True,India
4,India,AS,Specifier/ Influencer,Enterprise,NaN,consulting,97,Quotation or purchase consultation,multi-split,partner,LGEIL,less than 3 months,corporate / office,True,India


In [64]:
df_test['customer_nation'] = df_test['customer_country'].str.split('/').str[-1]

import pycountry

def validate_country(data):
    try:
        return pycountry.countries.lookup(data).name
    except LookupError:
        return pd.NA   # 국가이름 아닐시 nan으로 반환
df_test['customer_nation'] = df_test['customer_nation'].apply(validate_country)
df_test_country = df_test['customer_nation']
df_test['customer_country'] = df_test_country
df_test.head()

,customer_country,business_unit,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,customer_position,response_corporate,expected_timeline,business_area,is_converted,customer_nation
0,<NA>,ID,End Customer,Enterprise,NaN,consulting,3,NaN,NaN,none,LGESP,NaN,retail,True,<NA>
1,<NA>,IT,End Customer,SMB,NaN,NaN,23,Quotation or Purchase Consultation,NaN,none,LGEUS,NaN,transportation,True,<NA>
2,<NA>,ID,Specifier/ Influencer,SMB,NaN,information technology,138,Quotation or Purchase Consultation,hospital tv,manager,LGEGF,less than 3 months,hospital & health care,False,<NA>
3,<NA>,ID,NaN,Enterprise,NaN,sales,3,Quotation or Purchase Consultation,NaN,none,LGEUS,more than a year,retail,False,<NA>
4,<NA>,ID,End Customer,Enterprise,1.0,engineering,107,Quotation or Purchase Consultation,led signage,others,LGESP,less than 3 months,corporate / office,False,<NA>


In [66]:
print(df_test['customer_country'].count())
print(df_test['customer_country'].nunique())
print(df_test['customer_country'].unique())
print(df['customer_country'].count())
print(df['customer_country'].nunique())
print(df['customer_country'].unique())


792
48
[<NA> 'Brazil' 'India' 'United States' 'Egypt' 'Mexico' 'Philippines'
 'Hong Kong' 'Colombia' 'Germany' 'Singapore' 'Viet Nam' 'United Kingdom'
 'Israel' 'Argentina' 'Peru' 'Indonesia' 'Poland' 'Spain' 'Australia'
 'Chile' 'Algeria' 'South Africa' 'Ecuador' 'Saudi Arabia' 'Türkiye'
 'Puerto Rico' 'Italy' 'El Salvador' 'Thailand' 'Malaysia' 'Greece'
 'Malta' 'Canada' 'Guatemala' 'Panama' 'France' 'Austria' 'Nigeria'
 'Korea, Republic of' 'Curaçao' 'Afghanistan' 'Senegal' 'Lithuania'
 'Bahrain' 'Costa Rica' 'Kenya' 'Oman' 'Iraq']
59299
74
['Philippines' 'India' 'Nigeria' 'Saudi Arabia' 'Singapore' 'Brazil'
 'none' 'South Africa' 'United States' 'Colombia' 'Mexico' 'Ghana' 'Egypt'
 'Australia' 'Kenya' 'Indonesia' 'Oman' 'Pakistan' 'United Kingdom'
 'Guatemala' 'Panama' 'Canada' 'Bangladesh' 'Qatar' 'Afghanistan' 'Chile'
 'Türkiye' 'El Salvador' 'Jordan' 'Iraq' 'Israel' 'Sri Lanka' 'Portugal'
 'Uruguay' 'Peru' 'Germany' 'Romania' 'Jamaica' 'Hungary' 'Poland' 'Spain'
 'Argentina' 'Ec

In [19]:
grouped_df = df.groupby('customer_country')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
# 수가 적은 나라 수 데이터 합 578개 (약 1.1%)
print(grouped_df[grouped_df['is_converted'] <= 20].sum())
grouped_df[grouped_df['is_converted'] <= 20]

customer_country    SwedenKorea, Republic ofAnguillaParaguayMaltaC...
is_converted                                                      578
dtype: object


,customer_country,is_converted
134,Sweden,20
79,"Korea, Republic of",20
4,Anguilla,20
110,Paraguay,19
90,Malta,18
...,...,...
16,Belarus,1
91,Mauritania,1
65,Iceland,1
61,Haiti,1


In [21]:
# null값 n으로 치환. null값 6017개
print(df[df['customer_country'] =='none']['customer_country'].count())
df.loc[df['customer_country'].isnull() == True, 'customer_country'] = 'none'
print(df[df['customer_country'] =='none']['customer_country'].count())
# 데이터가 20개 이하인 나라 카테고리
low_num_country = grouped_df[grouped_df['is_converted'] <= 20]['customer_country']

# low_num_에 있는 단어를 'none'으로 치환하는 함수
def replace_words_with_n(text, word_list):
    if text in word_list.values:
        return 'none'
    else:
        return text

# 데이터프레임의 'customer_country' 열에 함수를 적용하여 치환
df['customer_country'] = df['customer_country'].apply(lambda x: replace_words_with_n(x, low_num_country))

print(df['customer_country'].count())
print(df['customer_country'].nunique())
print(df['customer_country'].unique())

6017
6017
59299
74
['Philippines' 'India' 'Nigeria' 'Saudi Arabia' 'Singapore' 'Brazil'
 'none' 'South Africa' 'United States' 'Colombia' 'Mexico' 'Ghana' 'Egypt'
 'Australia' 'Kenya' 'Indonesia' 'Oman' 'Pakistan' 'United Kingdom'
 'Guatemala' 'Panama' 'Canada' 'Bangladesh' 'Qatar' 'Afghanistan' 'Chile'
 'Türkiye' 'El Salvador' 'Jordan' 'Iraq' 'Israel' 'Sri Lanka' 'Portugal'
 'Uruguay' 'Peru' 'Germany' 'Romania' 'Jamaica' 'Hungary' 'Poland' 'Spain'
 'Argentina' 'Ecuador' 'Hong Kong' 'Malaysia' 'Japan' 'Kuwait' 'Ireland'
 'Albania' 'Greece' 'Algeria' 'Nicaragua' 'Italy' 'Netherlands'
 'Dominican Republic' 'France' 'Bolivia, Plurinational State of'
 'Puerto Rico' 'Croatia' 'Belgium' 'Venezuela, Bolivarian Republic of'
 'Maldives' 'Morocco' 'Switzerland' 'Honduras' 'Thailand' 'Viet Nam'
 'Costa Rica' 'China' 'Lebanon' 'Bulgaria' 'Bahrain'
 'Taiwan, Province of China' 'Serbia']


In [67]:
# null값 n으로 치환. null값 6017개
print(df_test[df_test['customer_country'] =='none']['customer_country'].count())
df_test.loc[df_test['customer_country'].isnull() == True, 'customer_country'] = 'none'
print(df_test[df_test['customer_country'] =='none']['customer_country'].count())
# 데이터가 20개 이하인 나라 카테고리
low_num_country = grouped_df[grouped_df['is_converted'] <= 20]['customer_country']

# low_num_에 있는 단어를 'none'으로 치환하는 함수
def replace_words_with_n(text, word_list):
    if text in word_list.values:
        return 'none'
    else:
        return text

# 데이터프레임의 'customer_country' 열에 함수를 적용하여 치환
df_test['customer_country'] = df_test['customer_country'].apply(lambda x: replace_words_with_n(x, low_num_country))

print(df_test['customer_country'].count())
print(df_test['customer_country'].nunique())
print(df_test['customer_country'].unique())

0
4479
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/heohyeonjun/anaconda3/envs/tf29_py39/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 153, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 182, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'customer_country'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/heohyeonjun/anaconda3/envs/tf29_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/09/9bnyqv215k92q_njqzl8zzbm0000gn/T/ipykernel_98036/2896356975.py", l

# business unit

In [22]:
print(df['business_unit'].nunique())
print(df['business_unit'].unique())

5
['AS' 'ID' 'IT' 'Solution' 'CM']


# customer_type

In [23]:
# 소문자 치환
df['customer_type'] = df['customer_type'].str.lower()
# other, others, etc를 other로 통합
df.loc[df['customer_type'].str.contains('other') == True,'customer_type'] = 'other'
df.loc[df['customer_type'].str.contains('etc') == True,'customer_type'] = 'other'
# end-customer, end-usr, end customer, commercial end-user을 end-customer로 치환
df.loc[df['customer_type'].str.contains('end') == True,'customer_type'] = 'end-customer'
print(df['customer_type'].nunique())
print(df['customer_type'].unique())

28
['end-customer' 'specifier/ influencer' 'service partner'
 'channel partner' nan 'corporate' 'specifier / influencer'
 'solution eco-partner' 'distributor' 'other' 'system integrator'
 'consultant' 'installer' 'homeowner' 'technician' 'installer/contractor'
 'engineer' 'manager / director' 'developer' 'architect/consultant'
 'hvac engineer' 'reseller' 'software/solution provider'
 'software / solution provider' 'dealer/distributor' 'technical assistant'
 'interior designer' 'home owner' 'administrator']


# enterprise

In [24]:
print(df['enterprise'].nunique())
print(df['enterprise'].unique())

2
['Enterprise' 'SMB']


# product_category

In [25]:
print(df['product_category'].count())
print(df['product_category'].nunique())
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
print(grouped_df[-30:])

39925
357
                                      product_category  is_converted
187                                    one quick works             1
182                                      not specified             1
181                           multi-split,single-split             1
172                              monitorindustrial_rmk             1
133                      monior/monitor tv,audio/video             1
134                          monior/monitor tv,chiller             1
136            monior/monitor tv,projector,audio/video             1
137                     monior/monitor tv,refrigerator             1
138         monior/monitor tv,refrigerator,audio/video             1
139  monior/monitor tv,system ac,tv,pc,refrigerator...             1
140  monior/monitor tv,system ac,tv,refrigerator,wa...             1
142  monior/monitor tv,tv,commercial tv,pc,refriger...             1
147          monitor signage,commercial tv,audio/video             1
149  monitor signage,com

In [26]:
# 소문자로 치환
df['product_category'] = df['product_category'].str.lower()

# null값 n으로 치환
df.loc[df['product_category'].isnull() == True, 'product_category'] = 'none'

# 기호들을 공백으로 치환
def replace_symbols(text):
    # 만약 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    return text.replace(',', ' ').replace('-', ' ').replace('/', ' ')

replaced_df = df['product_category'].apply(replace_symbols)
df['product_category'] = replaced_df
print(df['product_category'].count())
print(df['product_category'].nunique())
print(df['product_category'].unique())

59299
356
['multi split' 'single split' 'vrf' 'chiller' 'etc.' 'rac'
 'teto ou cassete inverter' 'none' 'software solution'
 'all lg vrf systems' 'ventilation' 'a thermodynamic water heater'
 'residential air conditioner' 'outros' 'heating' 'multi v5 vrf'
 'split tunggal' 'multi inverter' 'חימום' 'standard signage'
 'high brightness signage' 'interactive signage' 'video wall signage'
 'led signage' 'hotel tv' 'signage care solution' 'oled signage' 'idb'
 'video wall' 'one:quick series' 'commercial tv' 'lg one:quick'
 'special signage' 'other' 'accessories' 'commercial display'
 'interactive digital board' 'standard' 'uhd signage' 'smart tv signage'
 'lg one:quick series' 'pro:centric' 'ur640' 'ur640s' 'webos'
 'signage care solutions' 'aio | one quick' 'high brightness'
 'hospital tv' 'digital signage' 'monitor' 'pc' 'laptop' 'projector'
 'cloud device' 'medical display' 'washing machine dryer' 'solar chiller'
 'system ac rac' 'monitor signage commercial tv monior monitor tv'
 'monitor

In [ ]:
# import re

# # 알파벳을 포함하고, 한글, 중국어 등의 다른 문자를 포함하지 않은 데이터만 남기기
# # False가 반환되는 값은 'etc'로 치환
# def contains_valid_chars(text):
#     if bool(re.match(r'^[a-zA-Z\s]+$', str(text))):
#         return text
#     else:
#         return 'etc'

# # product_category 열에 함수 적용
# df['product_category'] = df['product_category'].apply(contains_valid_chars)

# # 출력
# # etc에 포함되는 데이터 4079개 (전체 데이터의 11%)
# print(df[df['product_category'] == 'etc']['product_category'].count())
# print(df['product_category'].count())
# print(df['product_category'])

In [27]:
# other, others, etc.를 other로 통합
df.loc[df['product_category'].str.contains('other') == True,'customer_type'] = 'other'
df.loc[df['product_category'].str.contains('etc') == True,'customer_type'] = 'other'

print(df['product_category'].count())
print(df['product_category'].nunique())
print(df['product_category'].unique())

59299
356
['multi split' 'single split' 'vrf' 'chiller' 'etc.' 'rac'
 'teto ou cassete inverter' 'none' 'software solution'
 'all lg vrf systems' 'ventilation' 'a thermodynamic water heater'
 'residential air conditioner' 'outros' 'heating' 'multi v5 vrf'
 'split tunggal' 'multi inverter' 'חימום' 'standard signage'
 'high brightness signage' 'interactive signage' 'video wall signage'
 'led signage' 'hotel tv' 'signage care solution' 'oled signage' 'idb'
 'video wall' 'one:quick series' 'commercial tv' 'lg one:quick'
 'special signage' 'other' 'accessories' 'commercial display'
 'interactive digital board' 'standard' 'uhd signage' 'smart tv signage'
 'lg one:quick series' 'pro:centric' 'ur640' 'ur640s' 'webos'
 'signage care solutions' 'aio | one quick' 'high brightness'
 'hospital tv' 'digital signage' 'monitor' 'pc' 'laptop' 'projector'
 'cloud device' 'medical display' 'washing machine dryer' 'solar chiller'
 'system ac rac' 'monitor signage commercial tv monior monitor tv'
 'monitor

In [28]:
# 그룹으로 묶어 카테고리별 데이터 수 확인
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
grouped_df[:30]

,product_category,is_converted
181,none,19374
98,interactive digital board,6154
314,vrf,5640
174,multi split,3676
309,video wall signage,2029
80,etc.,2018
114,led signage,1895
99,interactive signage,1862
234,single split,1574
212,rac,1447


In [29]:
grouped_df[-10:]

,product_category,is_converted
159,monitor signage mobile,1
161,monitor signage monior monitor tv commercial tv,1
162,monitor signage monior monitor tv pc tv,1
163,monitor signage monior monitor tv system ac va...,1
164,monitor signage monior monitor tv tv audio video,1
165,monitor signage monior monitor tv vacuum clean...,1
167,monitor signage solar,1
168,monitor signage solar monior monitor tv pc pro...,1
169,monitor signage solar robot water care,1
355,高亮度顯示屏,1


In [ ]:
# # monitor signage를 포함하는 데이터 통합 -> monitor signage가 제일 앞에 있는 경향
# # monior monitor tv를 포함하는 데이터 통합 -> 그 다음으로 많이 보임
# # commercial tv, vrf multi split -> 마찬가지
# df.loc[df['product_category'].str.contains('monitor signage') == True,'product_category'] = 'monitor signage'
# df.loc[df['product_category'].str.contains('monior monitor tv') == True,'product_category'] = 'monior monitor tv'
# df.loc[df['product_category'].str.contains('commercial tv') == True,'product_category'] = 'commercial tv'
# df.loc[df['product_category'].str.contains('vrf multi split') == True,'product_category'] = 'vrf multi split'
# print(df['product_category'].count())
# print(df['product_category'].nunique())
# print(df['product_category'].unique())

In [30]:
# 카테고리가 20개 이하인 카테고리 none으로 치환
# 모두 합하면 854개(약 2.1%)
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
print(grouped_df[grouped_df['is_converted'] <= 20].sum())
grouped_df[grouped_df['is_converted'] <= 20]

product_category    servicesmonitor & pccommercial tv tvaire acond...
is_converted                                                      854
dtype: object


,product_category,is_converted
228,services,20
144,monitor & pc,19
64,commercial tv tv,19
35,aire acondicionado residencial,18
192,onequick series,16
...,...,...
165,monitor signage monior monitor tv vacuum clean...,1
167,monitor signage solar,1
168,monitor signage solar monior monitor tv pc pro...,1
169,monitor signage solar robot water care,1


In [31]:
low_num_category = grouped_df[grouped_df['is_converted'] <= 20]['product_category']

# low_num_에 있는 단어를 'none'으로 치환하는 함수
# 데이터프레임의 'product_category' 열에 함수를 적용하여 치환
df['product_category'] = df['product_category'].apply(lambda x: replace_words_with_n(x, low_num_category))

print(df['product_category'].count())
print(df['product_category'].nunique())
print(df['product_category'].unique())

59299
60
['multi split' 'single split' 'vrf' 'chiller' 'etc.' 'rac'
 'teto ou cassete inverter' 'none' 'software solution'
 'all lg vrf systems' 'ventilation' 'a thermodynamic water heater'
 'residential air conditioner' 'outros' 'heating' 'multi inverter'
 'standard signage' 'high brightness signage' 'interactive signage'
 'video wall signage' 'led signage' 'hotel tv' 'signage care solution'
 'oled signage' 'idb' 'video wall' 'one:quick series' 'commercial tv'
 'special signage' 'accessories' 'commercial display'
 'interactive digital board' 'uhd signage' 'smart tv signage'
 'lg one:quick series' 'pro:centric' 'ur640' 'ur640s' 'webos'
 'high brightness' 'hospital tv' 'digital signage' 'monitor' 'pc' 'laptop'
 'projector' 'cloud device' 'medical display'
 'monitor signage monior monitor tv' 'solar ess' '43us660h0sd.awz'
 'control' 'multi v 5 air' 'ar condicionado residencial'
 'technical support' 'medical displays' 'virtual production'
 'ogrzewanie (pompy ciepła)' 'ledallinone' 'videow

# customer_position

In [32]:
import re
print(df['customer_position'].count())
# 소문자로 치환
df['customer_position'] = df['customer_position'].str.lower()
# other, others, etc.를 other로 통합
df.loc[df['customer_position'].str.contains('other') == True,'customer_position'] = 'other'
df.loc[df['customer_position'].str.contains('etc') == True,'customer_position'] = 'other'

# null값 n으로 치환. none과 null값은 다른것으로 판단
# null값 없음
df.loc[df['customer_position'].isnull() == True, 'customer_position'] = 'n'
print(df[df['customer_position'] =='none']['customer_position'].count())
print(df[df['customer_position'] =='n']['customer_position'].count())
# none이 아닌 데이터 약 4만개

# 기호들을 공백으로 치환
def replace_symbols(text):
    # 만약 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    text = text.replace(',', ' ').replace('-', ' ').replace('/', ' ')
    # 단어 사이의 공백이 두 칸 이상인 부분을 공백 한 칸으로 바꿈
    text = re.sub(r'\s{2,}', ' ', text)
    return text

replaced_df = df['customer_position'].apply(replace_symbols)
df['customer_position'] = replaced_df

print(df['customer_position'].nunique())
print(df['customer_position'].unique())

59299
19680
0
113
['entry level' 'ceo founder' 'partner' 'manager' 'vice president'
 'associate analyst' 'c level executive' 'none' 'director' 'other'
 'intern' 'trainee'
 'this is a consume display requirement for home purpose.' 'vicepresident'
 'consulting' 'entrylevel' 'c levelexecutive' 'unpaid' 'physics teacher'
 'assistant professor' 'av management' 'maths lecturer' 'founder'
 'engineering' 'installer' 'homeowner' 'consultant' 'commercial end user'
 'employee' 'bulgaria' 'lider de desarrollo' 'administrative'
 'entrepreneurship' 'decision influencer' 'decision maker' 'customer'
 'not applicable' 'no influence' 'commercial consultant' 'science teacher'
 'technical' 'architecture consult' 'architect consultant' 'exhibition'
 'hospital' 'end user' 'government' 'manufacturer'
 'software solution provider' 'system integrator'
 'medical device manufacturer' 'distributor' 'business unit director'
 'business development' 'operations' 'vp' 'cargo' 'guest faculty'
 'physics faculty' 'caree

In [ ]:
# # 특정 단어 수 이상인 문장 제거
# def replace_long_sentences(sentences, threshold):
#     updated_sentences = []
#     for sentence in sentences:
#         # 문장을 단어로 분할하여 단어의 개수를 세기
#         words = sentence.split()
#         if len(words) >= threshold:
#             # 'n'으로 치환
#             updated_sentences.append('none')
#         else:
#             updated_sentences.append(sentence)
#     return updated_sentences

# filtered_sentences = replace_long_sentences(df['customer_position'], 5)
# df['customer_position'] = filtered_sentences
# print(df['customer_position'].count())
# print(df['customer_position'].nunique())
# print(df['customer_position'].unique())

In [33]:
# 카테고리가 20개 이하인 카테고리 n으로 치환
# 10개 이하인 카테고리 252개 (약 0.6%)
grouped_df = df.groupby('customer_position')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
print(grouped_df[grouped_df['is_converted'] <= 20].sum())
grouped_df[grouped_df['is_converted'] <= 20]

customer_position    unpaiddecision influenceremployeedecision make...
is_converted                                                       252
dtype: object


,customer_position,is_converted
109,unpaid,17
32,decision influencer,16
40,employee,14
33,decision maker,12
111,vicepresident,11
...,...,...
66,maths lecturer,1
69,mindenes,1
70,neet olympiad expert faculty,1
77,pathologist,1


In [34]:
low_num_position = grouped_df[grouped_df['is_converted'] <= 20]['customer_position']

# low_num_position에 있는 단어를 'n'으로 치환하는 함수
# 데이터프레임의 'customer_position' 열에 함수를 적용하여 치환
df['customer_position'] = df['customer_position'].apply(lambda x: replace_words_with_n(x, low_num_position))

print(df['customer_position'].count())
print(df['customer_position'].nunique())
print(df['customer_position'].unique())

59299
22
['entry level' 'ceo founder' 'partner' 'manager' 'vice president'
 'associate analyst' 'c level executive' 'none' 'director' 'other'
 'intern' 'trainee' 'installer' 'homeowner' 'consultant' 'hospital'
 'end user' 'government' 'manufacturer' 'software solution provider'
 'system integrator' 'cargo']


# response_corporate

In [35]:
print(df['response_corporate'].nunique())
print(df['response_corporate'].unique())
grouped_df = df.groupby('response_corporate')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
grouped_df

53
['LGEPH' 'LGEIL' 'LGEAF' 'LGESJ' 'LGESL' 'LGESP' 'LGEGF' 'LGESA' 'LGEUS'
 'LGECB' 'LGEMS' 'LGEEG' 'LGEEF' 'LGEAP' 'LGEIN' 'LGEUK' 'LGEKR' 'LGEPS'
 'LGECI' 'LGECL' 'LGETK' 'LGELF' 'LGEPT' 'LGEPR' 'LGEDG' 'LGERO' 'LGEMK'
 'LGEPL' 'LGECZ' 'LGEES' 'LGEAR' 'LGEHK' 'LGEML' 'LGEJP' 'LGEHS' 'LGEAS'
 'LGEYK' 'LGEIS' 'LGEBN' 'LGEFS' 'LGESW' 'LGEMC' 'LGEAG' 'LGEEB' 'LGETH'
 'LGEVH' 'LGECH' 'LGELA' 'LGETT' 'LGERA' 'LGEUR' 'LGEIR' 'LGEBT']


,response_corporate,is_converted
21,LGEIL,16908
43,LGESP,9311
50,LGEUS,5955
32,LGEMS,2768
33,LGEPH,2651
18,LGEGF,2149
7,LGECB,2079
48,LGEUK,1651
41,LGESJ,1469
10,LGECL,1339


# expected_timeline

In [36]:
print(df['expected_timeline'].nunique())
print(df['expected_timeline'].unique()[:10])

449
['less than 3 months' nan '3 months ~ 6 months' '9 months ~ 1 year'
 'more than a year' '6 months ~ 9 months'
 'quote has been sent to customer.'
 'partner was exploring on eol model for some requiremeent. no designated sales or closure expected hence closing in the system.'
 'already close in july purchse with rd'
 'client not interested in product.. receing call and not answering properly']


In [ ]:
# # 특수문자를 단순 제거하면 안된다
# df['expected_timeline'] = df['expected_timeline'].str.lower()
# # 알파벳, 숫자, 공백을 제외한 모든 문자 제거하는 함수
# def remove_special_characters(text):
#     pattern = r'[^a-zA-Z0-9\s]'  # 알파벳과 숫자, 공백을 제외한 모든 문자
#     text = re.sub(pattern, '', text)
#     text = re.sub(r'\s{2,}', ' ', text)  # 단어 사이의 공백이 두 칸 이상인 부분을 공백 한 칸으로 바꿈
#     return text

# # 'expected_timeline' 열의 모든 값을 문자열로 변환하여 리스트로 가져옴
# expected_timeline = df['expected_timeline'].astype(str).tolist()

# # 각 문자열에 대해 특수 문자 제거 함수를 적용하여 변환된 값들을 리스트로 저장
# cleaned_expected_timeline = [remove_special_characters(text) for text in expected_timeline]

# # 변환된 값들을 다시 DataFrame의 'expected_timeline' 열에 할당
# df['expected_timeline'] = cleaned_expected_timeline

In [42]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/heohyeonjun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/heohyeonjun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/heohyeonjun/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
# 토큰화 & 불용어 제거
from nltk.corpus import stopwords
from konlpy.tag import Okt
from nltk.stem import PorterStemmer

# 불용어 제거를 위한 영어 불용어 리스트
stop_words = set(stopwords.words('english'))

# 어간 추출
stemmer = PorterStemmer()

# 텍스트 데이터
text = df['expected_timeline'].astype(str)

# 각 문장을 단어로 토큰화한 후 불용어를 제거하는 과정
tokenized_sentences = []
for sentence in text:
    # 문장을 단어로 토큰화
    tokens = word_tokenize(sentence)
    # 불용어 제거 & 어간 추출
    filtered_sentence = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    tokenized_sentences.append(filtered_sentence)

tokenized_sentences = pd.Series(tokenized_sentences)
df['expected_timeline'] = tokenized_sentences

In [ ]:
# 정수 인코딩
from sklearn.preprocessing import LabelEncoder

# 모든 리스트의 요소를 하나의 리스트로 펼치기
all_values = [val for sublist in tokenized_sentences for val in sublist]

# 정수 인코딩을 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
label_encoder.fit(all_values)

# 각 리스트의 요소를 정수로 변환하여 저장
df['expected_timeline'] = tokenized_sentences.apply(lambda x: [label_encoder.transform([item])[0] for item in x])

df['expected_timeline']
# 이거를 어떻게 이용해야 하니..

0                 [356, 50, 392]
1                 [356, 50, 392]
2                 [356, 50, 392]
3                 [356, 50, 392]
4                 [356, 50, 392]
                  ...           
59294    [50, 392, 658, 72, 392]
59295    [91, 392, 658, 18, 654]
59296             [356, 50, 392]
59297                      [654]
59298             [356, 50, 392]
Name: expected_timeline, Length: 59299, dtype: object

# business_area

In [44]:
print(df['business_area'].nunique())
print(df['business_area'].unique())

12
['corporate / office' nan 'education' 'hotel & accommodation'
 'hospital & health care' 'special purpose' 'residential (home)'
 'government department' 'retail' 'factory'
 'power plant / renewable energy' 'transportation' 'public facility']


## 2. 데이터 전처리

### 레이블 인코딩

In [45]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [ ]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

NameError: name 'df_train' is not defined

다시 학습 데이터와 제출 데이터를 분리합니다.

In [ ]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [46]:
x_train, x_val, y_train, y_val = train_test_split(
    df.drop("is_converted", axis=1),
    df["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모듈 불러오기

In [48]:
import catboost
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, cv
import os
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

### 카테고리 칼럼과 아닌 칼럼

In [51]:
cat_cols = df.columns[df.dtypes == 'object'].to_list() # 카테고리 칼럼
num_cols = df.columns[df.dtypes != 'object'].to_list() # 카테고리 칼럼이 아닌 수

In [54]:
print(len(cat_cols)+len(num_cols))

15


In [57]:
df.shape

(59299, 15)

In [58]:
df_test = 

NameError: name 'col' is not defined

### 모델 정의 

In [ ]:
model = DecisionTreeClassifier()

### 모델 학습

In [ ]:
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

### 모델 성능 보기

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  737   210]
 [  261 10652]]

정확도: 0.9603
정밀도: 0.7385
재현율: 0.7782
F1: 0.7578


## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [ ]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

1179

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**

In [ ]:
drop_feature = []
for idx, feature, in enumerate(df_train.columns.drop('is_converted')):
    # 중요도 낮은 feature 찾기
    print('{} : {}'.format(feature,model.feature_importances_[idx]))
    if (model.feature_importances_[idx] <= 0.01) == True:
        drop_feature.append(feature)

bant_submit : 0.018588446588475607
customer_country : 0.02553993748774244
business_unit : 0.005622663602754244
com_reg_ver_win_rate : 0.0371391613483653
customer_idx : 0.5329801355366556
customer_type : 0.04227665380506234
enterprise : 0.006818568396776413
historical_existing_cnt : 0.008404522625186891
id_strategic_ver : 0.0009845632378288533
it_strategic_ver : 0.0
idit_strategic_ver : 0.00045709854151916964
customer_job : 0.015391049506433208
lead_desc_length : 0.048908949434221395
inquiry_type : 0.007298900002791117
product_category : 0.018135810476922472
product_subcategory : 0.010163437499362596
product_modelname : 0.005867534892243128
customer_country.1 : 0.01945650274380273
customer_position : 0.014681639446527662
response_corporate : 0.01985071071880388
expected_timeline : 0.010665214923445025
ver_cus : 0.0007003204375270562
ver_pro : 0.0018727123320172027
ver_win_rate_x : 0.0028297618706202366
ver_win_ratio_per_bu : 0.007854946441456474
business_area : 0.005776700771319157
busi

In [ ]:
drop_feature

['business_unit',
 'enterprise',
 'historical_existing_cnt',
 'id_strategic_ver',
 'it_strategic_ver',
 'idit_strategic_ver',
 'inquiry_type',
 'product_modelname',
 'ver_cus',
 'ver_pro',
 'ver_win_rate_x',
 'ver_win_ratio_per_bu',
 'business_area']

In [ ]:
# 중요도 낮은 feature 제거
df_train = df_train.drop(drop_feature, axis = 1)

In [ ]:
df_train.head()

,bant_submit,customer_country,com_reg_ver_win_rate,customer_idx,customer_type,customer_job,lead_desc_length,product_category,product_subcategory,customer_country.1,customer_position,response_corporate,expected_timeline,business_subarea,lead_owner,is_converted
0,1.0,9070,0.066667,32160,10,420,62,180,349,9070,47,33,246,28,0,True
1,1.0,8406,0.066667,23122,10,303,96,180,349,8406,21,33,246,0,1,True
2,1.0,6535,0.088889,1755,10,160,56,238,349,6535,80,21,246,17,2,True
3,1.0,3388,0.088889,4919,10,166,44,318,349,3388,21,21,246,44,3,True
4,1.0,5799,0.088889,17126,29,84,97,180,349,5799,80,21,246,86,4,True


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bant_submit           59299 non-null  float64
 1   customer_country      59299 non-null  int64  
 2   com_reg_ver_win_rate  14568 non-null  float64
 3   customer_idx          59299 non-null  int64  
 4   customer_type         59299 non-null  int64  
 5   customer_job          59299 non-null  int64  
 6   lead_desc_length      59299 non-null  int64  
 7   product_category      59299 non-null  int64  
 8   product_subcategory   59299 non-null  int64  
 9   customer_country.1    59299 non-null  int64  
 10  customer_position     59299 non-null  int64  
 11  response_corporate    59299 non-null  int64  
 12  expected_timeline     59299 non-null  int64  
 13  business_subarea      59299 non-null  int64  
 14  lead_owner            59299 non-null  int64  
 15  is_converted       

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
)

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

In [ ]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  796   205]
 [  210 10649]]

정확도: 0.9650
정밀도: 0.7913
재현율: 0.7952
F1: 0.7932


In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)